In [56]:
import pandas as pd
from pandas.api.types import is_numeric_dtype
from tqdm import tqdm

In [ ]:
frequency = 'monthly'
column = 'ridership'
years = [2020,2021,2022,2023]

In [58]:
nyc_master_df = pd.read_csv(f'Data/Master Sets/NYC Master/Yearly Combined/NYC_Combined_{frequency}.csv')

In [59]:
nyc_master_df.head()

,Unnamed: 0,year,month,day,station_complex_id,station_complex,borough,latitude,longitude,ridership,...,Structure,GTFS Latitude,GTFS Longitude,North Direction Label,South Direction Label,ADA,ADA Northbound,ADA Southbound,ADA Notes,Georeference
0,0,2020,10,1,1,"Astoria-Ditmars Blvd (N,W)",Queens,40.775036,-73.912030,5980.0,...,Elevated,40.775036,-73.912034,Last Stop,Manhattan,0.0,0.0,0.0,NaN,POINT (-73.912034 40.775036)
1,1,2020,10,1,10,"49 St (N,R,W)",Manhattan,40.759900,-73.984140,4148.0,...,Subway,40.759901,-73.984139,Uptown,Downtown,2.0,1.0,0.0,Uptown only,POINT (-73.984139 40.759901)
2,2,2020,10,1,100,"Hewes St (M,J)",Brooklyn,40.706870,-73.953430,1330.0,...,Elevated,40.706870,-73.953431,Outbound,Manhattan,0.0,0.0,0.0,NaN,POINT (-73.953431 40.70687)
3,3,2020,10,1,101,"Marcy Av (M,J,Z)",Brooklyn,40.708360,-73.957756,5406.0,...,Elevated,40.708359,-73.957757,Outbound,Manhattan,1.0,1.0,1.0,NaN,POINT (-73.957757 40.708359)
4,4,2020,10,1,103,"Bowery (J,Z)",Manhattan,40.720280,-73.993910,1179.0,...,Subway,40.720280,-73.993915,Brooklyn,Downtown,0.0,0.0,0.0,NaN,POINT (-73.993915 40.72028)


In [60]:
nyc_master_df = nyc_master_df[nyc_master_df['year'].isin(years)]

In [61]:
station_network_list = nyc_master_df['GTFS Stop ID'].unique()
station_network_list.sort()
time_interval = nyc_master_df['unique_time'].unique()
time_interval.sort()

In [62]:
def compute_time_series(col: str):

    if col not in nyc_master_df:
        raise ValueError('Column does not exist')
    temp_dict = {}

    if is_numeric_dtype(nyc_master_df[col]):
        print('is numeric')
    else:
        print('is not numeric')

    for station in tqdm(station_network_list):
        for time in time_interval:
            temp_df = nyc_master_df[(nyc_master_df['GTFS Stop ID'] == station) & (nyc_master_df['unique_time'] == time)]
            if is_numeric_dtype(temp_df[col]):
                val = temp_df[col].mean() # Takes mean of all rows
            else:
                val = temp_df[col].max()
            if station not in temp_dict:
                temp_dict[station] = {}
            temp_dict[station][time] = val
    return temp_dict

In [63]:
ts_matrix = pd.DataFrame(compute_time_series(column))

is numeric


100%|██████████| 426/426 [7:56:53<00:00, 67.17s/it]  


In [64]:
ts_matrix

,101,103,104,106,107,108,109,110,111,113,...,R41,R42,R43,R44,R45,S01,S03,S04,S30,S31
2020-07-01,2308.0,1183.0,2678.0,1684.0,409.0,2011.0,2333.0,391.0,3996.0,2819.0,...,3521.0,1945.0,1340.0,2970.0,1459.0,1254.0,345.0,2948.0,252.0,2438.0
2020-07-02,2274.0,1274.0,2824.0,1722.0,455.0,1910.0,2315.0,437.0,4233.0,2897.0,...,3595.0,1874.0,1350.0,3181.0,1575.0,1341.0,437.0,1631.5,292.0,2338.0
2020-07-03,1706.0,857.0,2246.0,1429.0,347.0,1739.0,2019.0,385.0,3566.0,2436.0,...,2748.0,1505.0,957.0,2377.0,1043.0,1158.0,311.0,1336.5,195.0,1114.0
2020-07-04,1151.0,643.0,1537.0,1085.0,231.0,1287.0,1572.0,299.0,2209.0,1931.0,...,1712.0,1096.0,624.0,1447.0,678.0,944.0,271.0,1004.5,85.0,673.0
2020-07-05,989.0,613.0,1291.0,908.0,257.0,1149.0,1426.0,250.0,1926.0,1630.0,...,1625.0,1000.0,503.0,1416.0,629.0,834.0,223.0,823.5,69.0,518.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-27,3827.0,2246.0,4853.0,2606.0,996.0,3255.0,3276.0,3855.0,5784.0,5549.0,...,7383.0,4464.0,3062.0,8340.0,3147.0,2689.0,768.0,6688.0,337.0,5313.0
2023-12-28,3920.0,2440.0,4876.0,2647.0,1056.0,3297.0,3643.0,4072.0,5949.0,5881.0,...,7424.0,4572.0,3275.0,9115.0,3300.0,2835.0,769.0,7169.0,374.0,5292.0
2023-12-29,3889.0,2553.0,5241.0,2744.0,1101.0,3546.0,3883.0,4144.0,5955.0,6146.0,...,7840.0,4591.0,3373.0,10189.0,3262.0,3319.0,854.0,3879.0,382.0,5667.0
2023-12-30,2665.0,1806.0,3536.0,2095.0,676.0,2632.0,2919.0,3118.0,4626.0,4843.0,...,5451.0,3539.0,2469.0,7451.0,2146.0,2493.0,640.0,3240.5,272.0,2632.0


In [65]:
# Shortcut solution to set all NA values to 0
ts_matrix.fillna(0, inplace=True)

# TODO Imputation for NA values

In [66]:
ts_matrix.to_csv(f'Data/Master Sets/NYC Master/NYC_Time_Series_{frequency}_{column}.csv')